In [1]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
import torch

# 데이터셋 로드 및 준비
dataset = load_dataset("your_custom_dataset")

# 모델 및 프로세서 로드
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# 데이터 로더 생성
class CustomDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        audio = self.dataset[idx]["audio"]
        transcription = self.dataset[idx]["transcription"]
        
        inputs = self.processor(audio, sampling_rate=16000, return_tensors="pt")
        labels = self.processor(transcription, return_tensors="pt").input_ids
        
        return inputs.input_values, labels

data_loader = DataLoader(CustomDataset(dataset, processor), batch_size=16, shuffle=True)

# 파인튜닝
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(5):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_values, labels = batch
        input_values, labels = input_values.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(input_values, labels=labels)
        loss = outputs.loss
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}")

# 모델 저장
torch.save(model.state_dict(), "fine_tuned_whisper_model.pth")


c:\Users\user\miniconda3\envs\conda311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetNotFoundError: Dataset 'your_custom_dataset' doesn't exist on the Hub or cannot be accessed.

In [ ]:
# https://www.kaggle.com/datasets/bryanpark/korean-single-speaker-speech-dataset
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bryanpark/korean-single-speaker-speech-dataset")

print("Path to dataset files:", path)

In [2]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

# 모델 및 프로세서 로드
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# 데이터 로더 생성
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        audio = self.dataset[idx]["audio"]
        transcription = self.dataset[idx]["transcription"]
        
        inputs = self.processor(audio, sampling_rate=16000, return_tensors="pt")
        labels = self.processor(transcription, return_tensors="pt").input_ids
        
        return inputs.input_values, labels

# 데이터셋 로드 및 파인튜닝
# KEBAP 또는 Korean Speech mixture with Noise Dataset을 사용
data_loader = torch.utils.data.DataLoader(CustomDataset(dataset, processor), batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(5):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_values, labels = batch
        input_values, labels = input_values.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(input_values, labels=labels)
        loss = outputs.loss
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}")

# 모델 저장
torch.save(model.state_dict(), "fine_tuned_whisper_model.pth")


c:\Users\user\miniconda3\envs\conda311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--openai--whisper-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


NameError: name 'dataset' is not defined